In [2]:
pip install DeepXDE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.8/179.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import deepxde as dde
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import pandas as pd
from IPython.display import display


x_lower = -25
x_upper = 25
t_lower = 0
t_upper = 1

α = 0
β = 0

α_0 = 0
k = 1
ω = 0.88
x_0 = 0
θ_0 = 0
μ = 4 * (k ** 2 - ω)

x = np.linspace(x_lower, x_upper, 256)
t = np.linspace(t_lower, t_upper, 201)
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

space_domain = dde.geometry.Interval(x_lower, x_upper)
time_domain = dde.geometry.TimeDomain(t_lower, t_upper)
geomtime = dde.geometry.GeometryXTime(space_domain, time_domain)


def pde(x, y):
    u = y[:, 0:1]
    v = y[:, 1:2]

    u_t = dde.grad.jacobian(y, x, i=0, j=1)
    v_t = dde.grad.jacobian(y, x, i=1, j=1)

    u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)

    f_u = -v_t + u_xx + u * (u ** 2 + v ** 2) * (1 - α * (u ** 2 + v ** 2) + β * (u ** 2 + v ** 2) ** 2)
    f_v = u_t + v_xx + v * (u ** 2 + v ** 2) * (1 - α * (u ** 2 + v ** 2) + β * (u ** 2 + v ** 2) ** 2)

    return [f_u, f_v]


bc_u_0 = dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=0
)
bc_u_1 = dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=0
)
bc_v_0 = dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=0, component=1
)
bc_v_1 = dde.icbc.PeriodicBC(
    geomtime, 0, lambda _, on_boundary: on_boundary, derivative_order=1, component=1
)


def init_cond_u(x):
    return np.cos(k * x[:, 0:1] + θ_0) * np.sqrt(μ * np.exp((x[:, 0:1] - x_0) * np.sqrt(μ)) / (
                ((1 / 2) * np.exp((x[:, 0:1] - x_0) * np.sqrt(μ)) + 1) ** 2 - (α_0 * μ / 3) * np.exp(
            2 * (x[:, 0:1] - x_0) * np.sqrt(μ))))


def init_cond_v(x):
    return np.sin(k * x[:, 0:1] + θ_0) * np.sqrt(μ * np.exp((x[:, 0:1] - x_0) * np.sqrt(μ)) / (
                ((1 / 2) * np.exp((x[:, 0:1] - x_0) * np.sqrt(μ)) + 1) ** 2 - (α_0 * μ / 3) * np.exp(
            2 * (x[:, 0:1] - x_0) * np.sqrt(μ))))


ic_u = dde.icbc.IC(geomtime, init_cond_u, lambda _, on_initial: on_initial, component=0)
ic_v = dde.icbc.IC(geomtime, init_cond_v, lambda _, on_initial: on_initial, component=1)


data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_u_0, bc_u_1, bc_v_0, bc_v_1, ic_u, ic_v],
    num_domain=10000,
    num_boundary=20,
    num_initial=200,
    train_distribution="pseudo",
)

net = dde.nn.FNN([2] + [32] * 2 + [2], "sin", "Glorot normal")

model = dde.Model(data, net)

model.compile("adam", lr=1e-3, loss="MSE")
model.train(iterations=10000, display_every=1000)

prediction = model.predict(X_star)

u = griddata(X_star, prediction[:, 0], (X, T), method="cubic")
v = griddata(X_star, prediction[:, 1], (X, T), method="cubic")

q = np.sqrt(u ** 2 + v ** 2)


def u_exact(x, t):
    return np.cos(k * x - ω * t + θ_0) * np.sqrt(μ * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) / (
                ((1 / 2) * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) + 1) ** 2 - (α_0 * μ / 3) * np.exp(
            2 * (x - 2 * k * t - x_0) * np.sqrt(μ))))


def v_exact(x, t):
    return np.sin(k * x - ω * t + θ_0) * np.sqrt(μ * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) / (
            ((1 / 2) * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) + 1) ** 2 - (α_0 * μ / 3) * np.exp(
        2 * (x - 2 * k * t - x_0) * np.sqrt(μ))))


def q_exact(x, t):
    return np.sqrt(μ * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) / (
            ((1 / 2) * np.exp((x - 2 * k * t - x_0) * np.sqrt(μ)) + 1) ** 2 - (α_0 * μ / 3) * np.exp(
        2 * (x - 2 * k * t - x_0) * np.sqrt(μ))))


data_ = {'x': X.flatten(), 't': T.flatten(), 'pred_u': u.flatten(), 'pred_v': v.flatten(), 'pred_h': q.flatten(),
        'true_u': u_exact(X, T).flatten(), 'true_v': v_exact(X, T).flatten(), 'true_h': q_exact(X, T).flatten()
         }

df = pd.DataFrame(data_)
df.to_orc('my_data.orc')
display(df)

fig, ax = plt.subplots(3, 2)

plt.suptitle("Results", fontsize=15, x=0.5, y=0.95)

ax[0, 0].set_ylabel("Real part")
ax[0, 0].imshow(
    u.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[1, 0].set_ylabel("Imaginary part")
ax[1, 0].imshow(
    v.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[2, 0].set_ylabel("Amplitude")
ax[2, 0].imshow(
    q.T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)

ax[0, 1].set_ylabel("Exact real part")
ax[0, 1].imshow(
    u_exact(X, T).T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[1, 1].set_ylabel("Exact imaginary part")
ax[1, 1].imshow(
    v_exact(X, T).T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)
ax[2, 1].set_ylabel("Exact amplitude")
ax[2, 1].imshow(
    q_exact(X, T).T,
    interpolation="nearest",
    cmap="viridis",
    extent=[t_lower, t_upper, x_lower, x_upper],
    origin="lower",
    aspect="auto",
)

plt.subplots_adjust(wspace=0.5, hspace=0.5)

plt.show()

No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Compiling model...
Building feed-forward neural network...
'build' took 0.076302 s



/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 2.102008 s

Training model...

Step      Train loss                                                                          Test loss                                                                           Test metric
0         [3.44e+00, 3.06e+00, 1.63e-02, 5.58e-04, 1.13e+01, 3.75e-07, 7.88e-01, 5.16e-01]    [3.44e+00, 3.06e+00, 1.63e-02, 5.58e-04, 1.13e+01, 3.75e-07, 7.88e-01, 5.16e-01]    []  
1000      [1.07e-03, 2.46e-03, 6.50e-06, 3.94e-06, 3.47e-05, 1.63e-05, 9.33e-03, 5.13e-03]    [1.07e-03, 2.46e-03, 6.50e-06, 3.94e-06, 3.47e-05, 1.63e-05, 9.33e-03, 5.13e-03]    []  
2000      [7.32e-04, 7.33e-04, 2.96e-06, 1.87e-05, 2.47e-05, 9.77e-07, 2.19e-03, 1.80e-03]    [7.32e-04, 7.33e-04, 2.96e-06, 1.87e-05, 2.47e-05, 9.77e-07, 2.19e-03, 1.80e-03]    []  
3000      [3.77e-04, 3.87e-04, 1.24e-05, 1.01e-05, 9.38e-06, 6.69e-06, 3.34e-04, 3.11e-04]    [3.77e-04, 3.87e-04, 1.24e-05, 1.01e-05, 9.38e-06, 6.69e-06, 3.34e-04, 3.11e-04]    []  
4000      [2.41e-04, 2.54e-04, 1

x    t    pred_u    pred_v    pred_h    true_u    true_v  \
0     -25.000000  0.0 -0.001866 -0.003106  0.003623  0.000119  0.000016   
1     -24.803922  0.0 -0.001320 -0.002382  0.002723  0.000122  0.000042   
2     -24.607843  0.0 -0.000889 -0.001268  0.001549  0.000119  0.000069   
3     -24.411765  0.0 -0.000612  0.000096  0.000620  0.000111  0.000097   
4     -24.215686  0.0 -0.000519  0.001534  0.001620  0.000096  0.000125   
...          ...  ...       ...       ...       ...       ...       ...   
51451  24.215686  1.0 -0.001408  0.003146  0.003446 -0.000141 -0.000614   
51452  24.411765  1.0 -0.001398  0.003728  0.003982 -0.000018 -0.000588   
51453  24.607843  1.0 -0.001196  0.004278  0.004442  0.000091 -0.000543   
51454  24.803922  1.0 -0.000776  0.004778  0.004841  0.000182 -0.000481   
51455  25.000000  1.0 -0.000102  0.005203  0.005204  0.000254 -0.000407   

         true_h  
0      0.000120  
1      0.000129  
2      0.000138  
3      0.000147  
4      0.000158  
...         ...  
51451  0.000630  
51452  0.000589  
51453  0.000550  
51454  0.000514  
51455  0.000480  

[51456 rows x 8 columns]